In [8]:
import itertools

class Z2DomainError(ValueError):
    pass

class Z2Field:
    def __init__(self, arg: 'int|Z2Field'):
        if isinstance(arg, int):
            self.n = arg % 5
        elif isinstance(arg, Z2Field):
            self.n = arg.n
        else:
            raise Z2DomainError("You are trying to create Z2Field from " + repr(other))
    
    def __add__(self, other):
        if isinstance(other, Z2Field):
            return Z2Field(self.n + other.n)
        elif isinstance(other, int):
            return Z2Field(self.n + (other % 5))
        else:
            raise Z2DomainError("You are trying to add " + repr(arg) + " to Z2Field")
        
    def __radd__(self, other):
        return self.__add__(other)
    
    def __mul__(self, other):
        if isinstance(other, Z2Field):
            return Z2Field(self.n * other.n)
        elif isinstance(other, int):
            return Z2Field(self.n * (other % 5))
        else:
            raise Z2DomainError("You are trying to multiply Z2Field by " + repr(other))
            
    def __divmod__(self, other):
        def div(a, b):
            for i in range(5):
                if  b * i % 5 == a:
                    return b
        if isinstance(other, Z2Field):
            return Z2Field(div(self.n, other.n)), Z2Field(0)
        elif isinstance(other, int):
            return Z2Field(div(self.n, (other % 5))), Z2Field(0)
        else:
            raise Z2DomainError("You are trying to divide Z2Field by " + repr(other))
            
    def __floordiv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[0]
    
    def __truediv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[0]

    def __mod__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[1]
    

    def __rfloordiv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[0]
    
    def __rtruediv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[0]

    def __rmod__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[1]
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __eq__(self, other):
        if isinstance(other, Z2Field):
            return self.n == other.n
        elif isinstance(other, int):
            return self.n == (other % 5)
        else:
            raise Z2DomainError("You are trying to compare Z2Field to " + repr(other))
    
    def __neg__(self):
        return Z2Field(5 - self.n)
    
    def __sub__(self, other):
        return self.__add__(Z2Field(other).__neg__())
    
    def __int__(self):
        return self.n
    
    def __str__(self):
        return 'Z' + str(self.n)
    


class PolynomialDomainError(ValueError):
    pass


class Polynomial:
    """Многочлен над полем"""
    
    def __init__(self, arg: 'int|list[int]|Polynomial|Z2Field'= 0):
        """
        coefficients -- коэффициенты
        """
        if isinstance(arg, int) or isinstance(arg, Z2Field):
            self.coefficients = [arg]
        elif isinstance(arg, list):
            self.coefficients = arg.copy()
        elif isinstance(arg, Polynomial):
            self.coefficients = arg.coefficients.copy()
        else:
            raise PolynomialDomainError("You are trying to create Polynomial from " + repr(arg))
        self.cleanup(self.coefficients)
            
    @staticmethod
    def cleanup(coefficients: 'list[int]') -> 'None':
        while len(coefficients) and coefficients[-1] == 0:
            del coefficients[-1]


    def __str__(self):
        return (" + ".join([
            str(c) + ("*x^" + str(i) if i > 0 else "")
            for c, i in reversed(list(zip(self.coefficients, itertools.count()))) if c != 0
        ])) if len(self.coefficients) else '0'

    
    def __eq__(self, other: 'Polynomial|int') -> 'bool':
        if isinstance(other, int):
            other = Polynomial(other)
        
        if isinstance(other, Polynomial):
            return self.coefficients == other.coefficients
        else:
            raise PolynomialDomainError("Can't say if Polynomial is equal to " + str(type(other)))

    
    def __lshift__(self, deg: 'int') -> 'Polynomial':
        return Polynomial(([0] * deg) + self.coefficients)
    
    def __add__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)
        
        sc = self.coefficients.copy()
        oc = other.coefficients.copy()
        
        # make lengths equal
        sc += [0] * (len(oc)-len(sc))
        oc += [0] * (len(sc)-len(oc))
        
        return Polynomial([
            (sce + oce) for sce, oce in zip(sc, oc)
        ])
        
        
    def __radd__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__add__(other)  # Коммутативность

    
    def __neg__(self) -> 'Polynomial':
        return Polynomial([-c for c in self.coefficients])

    
    def __sub__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)

        return self.__add__(other.__neg__())

    
    def __rsub__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__neg__().__add__(other)

    
    def __mul__(self, other: 'Polynomial|int') -> 'Polynomial':
        if not isinstance(other, Polynomial):
            other = Polynomial(other)
        
        c = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for sc, sci in zip(self.coefficients, itertools.count()):
            for oc, oci in zip(other.coefficients, itertools.count()):
                c[sci + oci] = (c[sci + oci] + (sc * oc))
        
        return Polynomial(c)
        
        
    def __rmul__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__mul__(other)  # Коммутативность

    
    def __divmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)

        sc = Polynomial(self)
        oc = other
        d = Polynomial()
        
        while len(sc.coefficients) >= len(oc.coefficients) > 0:
            c = sc.coefficients[-1] / oc.coefficients[-1]
            t = (len(sc.coefficients) - len(oc.coefficients))
            sc -= (oc << t) * c
            d += Polynomial([0] * t + [c])
            
        return d, sc
    

    def __floordiv__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(self, other)[0]

    def __mod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(self, other)[1]
    

    def __rfloordiv__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(other, self)[0]

    def __rmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(other, self)[1]
    
    
    def __rdivmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return Polynomial(other).__divmod__(self)
    

    def __int__(self) -> 'int':
        if not len(self.coefficients):
            return 0
        elif len(self.coefficients) == 1:
            return complex(self.coefficients[0])
        else:
            raise PolynomialDomainError("Can't consider higher degree Polynomial as an integer")
        
        return int(self.coefficients[0])

Z0 = Z2Field(0)
Z1 = Z2Field(1)

    
a = Polynomial([Z1,Z0,Z0,Z1,Z1,Z0][::-1])
b = Polynomial([Z1,Z1,Z1][::-1])


for q in range(0, 5):

    z = Polynomial([Z2Field(1), Z2Field(q), Z2Field(0), Z2Field(1)])
    a_c = 0

    for o in range(0, 5):
        for p in range(0, 5):
            for a in range(0, 5):
                for s in range(0, 5):
                    x = Polynomial([Z2Field(o), Z2Field(p), Z2Field(a), Z2Field(s)])
                    if ((z % x) == Polynomial(Z2Field(0)) and len(x.coefficients) > 1 and len((z // x).coefficients) > 1):
                        if (a_c == 0):
                            print(x, z // x, q)
                        a_c += 1
    if a_c == 0:
        print(z)


                            



Z1*x^1 + Z1 Z1*x^2 + Z4*x^1 + Z1 0
Z1*x^3 + Z1*x^1 + Z1
Z1*x^3 + Z2*x^1 + Z1
Z3*x^2 + Z1*x^1 + Z1 Z2*x^1 + Z1 3
Z2*x^2 + Z1*x^1 + Z1 Z3*x^1 + Z1 4


In [33]:
import itertools

class Z2DomainError(ValueError):
    pass

class Z2Field:
    def __init__(self, arg: 'int|Z2Field'):
        if isinstance(arg, int):
            self.n = arg % 5
        elif isinstance(arg, Z2Field):
            self.n = arg.n
        else:
            raise Z2DomainError("You are trying to create Z2Field from " + repr(other))
    
    def __add__(self, other):
        if isinstance(other, Z2Field):
            return Z2Field(self.n + other.n)
        elif isinstance(other, int):
            return Z2Field(self.n + (other % 5))
        else:
            raise Z2DomainError("You are trying to add " + repr(arg) + " to Z2Field")
        
    def __radd__(self, other):
        return self.__add__(other)
    
    def __mul__(self, other):
        if isinstance(other, Z2Field):
            return Z2Field(self.n * other.n)
        elif isinstance(other, int):
            return Z2Field(self.n * (other % 5))
        else:
            raise Z2DomainError("You are trying to multiply Z2Field by " + repr(other))
            
    def __divmod__(self, other):
        def div(a, b):
            for i in range(5):
                if  b * i % 5 == a:
                    return b
        if isinstance(other, Z2Field):
            return Z2Field(div(self.n, other.n)), Z2Field(0)
        elif isinstance(other, int):
            return Z2Field(div(self.n, (other % 5))), Z2Field(0)
        else:
            raise Z2DomainError("You are trying to divide Z2Field by " + repr(other))
            
    def __floordiv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[0]
    
    def __truediv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[0]

    def __mod__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[1]
    

    def __rfloordiv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[0]
    
    def __rtruediv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[0]

    def __rmod__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[1]
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __eq__(self, other):
        if isinstance(other, Z2Field):
            return self.n == other.n
        elif isinstance(other, int):
            return self.n == (other % 5)
        else:
            raise Z2DomainError("You are trying to compare Z2Field to " + repr(other))
    
    def __neg__(self):
        return Z2Field(5 - self.n)
    
    def __sub__(self, other):
        return self.__add__(Z2Field(other).__neg__())
    
    def __int__(self):
        return self.n
    
    def __str__(self):
        return 'Z' + str(self.n)
    


class PolynomialDomainError(ValueError):
    pass


class Polynomial:
    """Многочлен над полем"""
    
    def __init__(self, arg: 'int|list[int]|Polynomial|Z2Field'= 0):
        """
        coefficients -- коэффициенты
        """
        if isinstance(arg, int) or isinstance(arg, float):
            self.coefficients = [arg]
        elif isinstance(arg, list):
            self.coefficients = arg.copy()
        elif isinstance(arg, Polynomial):
            self.coefficients = arg.coefficients.copy()
        else:
            raise PolynomialDomainError("You are trying to create Polynomial from " + repr(arg))
        self.cleanup(self.coefficients)

    @staticmethod
    def cleanup(coefficients: 'list[Number]') -> 'None':
        epsilon = 1e-14
        while len(coefficients) and abs(coefficients[-1]) < epsilon:
            del coefficients[-1]


    def __str__(self):
        return (" + ".join([
            str(c) + ("*x^" + str(i) if i > 0 else "")
            for c, i in reversed(list(zip(self.coefficients, itertools.count()))) if c != 0
        ])) if len(self.coefficients) else '0'

    
    def __eq__(self, other: 'Polynomial|int') -> 'bool':
        if isinstance(other, int):
            other = Polynomial(other)
        
        if isinstance(other, Polynomial):
            return self.coefficients == other.coefficients
        else:
            raise PolynomialDomainError("Can't say if Polynomial is equal to " + str(type(other)))

    
    def __lshift__(self, deg: 'int') -> 'Polynomial':
        return Polynomial(([0] * deg) + self.coefficients)
    
    def __add__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)
        
        sc = self.coefficients.copy()
        oc = other.coefficients.copy()
        
        # make lengths equal
        sc += [0] * (len(oc)-len(sc))
        oc += [0] * (len(sc)-len(oc))
        
        return Polynomial([
            (sce + oce) for sce, oce in zip(sc, oc)
        ])
        
        
    def __radd__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__add__(other)  # Коммутативность

    
    def __neg__(self) -> 'Polynomial':
        return Polynomial([-c for c in self.coefficients])

    
    def __sub__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)

        return self.__add__(other.__neg__())

    
    def __rsub__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__neg__().__add__(other)

    
    def __mul__(self, other: 'Polynomial|int') -> 'Polynomial':
        if not isinstance(other, Polynomial):
            other = Polynomial(other)
        
        c = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for sc, sci in zip(self.coefficients, itertools.count()):
            for oc, oci in zip(other.coefficients, itertools.count()):
                c[sci + oci] = (c[sci + oci] + (sc * oc))
        
        return Polynomial(c)
        
        
    def __rmul__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__mul__(other)  # Коммутативность

    
    def __divmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)

        sc = Polynomial(self)
        oc = other
        d = Polynomial()
        
        while len(sc.coefficients) >= len(oc.coefficients) > 0:
            c = sc.coefficients[-1] / oc.coefficients[-1]
            t = (len(sc.coefficients) - len(oc.coefficients))
            sc -= (oc << t) * c
            d += Polynomial([0] * t + [c])
            
        return d, sc
    

    def __floordiv__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(self, other)[0]

    def __mod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(self, other)[1]
    

    def __rfloordiv__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(other, self)[0]

    def __rmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(other, self)[1]
    
    
    def __rdivmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return Polynomial(other).__divmod__(self)
    

    def __int__(self) -> 'int':
        if not len(self.coefficients):
            return 0
        elif len(self.coefficients) == 1:
            return complex(self.coefficients[0])
        else:
            raise PolynomialDomainError("Can't consider higher degree Polynomial as an integer")
        
        return int(self.coefficients[0])

a = Polynomial([-6, 1, 7, 6, 4])
b = Polynomial([-2, -3, 4, 2, 3])

def gcd(a,b):
    while len(b.coefficients) > 2:
        a, b = b, a % b
    return a
t = gcd(b,a)
print(t)
print(a % t, '\n', b % t)

                            



-1.0*x^2 + -1.0*x^1 + -2.0
0 
 0


In [39]:
import itertools

class Z2DomainError(ValueError):
    pass

class Z2Field:
    def __init__(self, arg: 'int|Z2Field'):
        if isinstance(arg, int):
            self.n = arg % 2
        elif isinstance(arg, Z2Field):
            self.n = arg.n
        else:
            raise Z2DomainError("You are trying to create Z2Field from " + repr(other))
    
    def __add__(self, other):
        if isinstance(other, Z2Field):
            return Z2Field(self.n + other.n)
        elif isinstance(other, int):
            return Z2Field(self.n + (other % 2))
        else:
            raise Z2DomainError("You are trying to add " + repr(arg) + " to Z2Field")
        
    def __radd__(self, other):
        return self.__add__(other)
    
    def __mul__(self, other):
        if isinstance(other, Z2Field):
            return Z2Field(self.n * other.n)
        elif isinstance(other, int):
            return Z2Field(self.n * (other % 2))
        else:
            raise Z2DomainError("You are trying to multiply Z2Field by " + repr(other))
            
    def __divmod__(self, other):
        if isinstance(other, Z2Field):
            return Z2Field(self.n // other.n), Z2Field(0)
        elif isinstance(other, int):
            return Z2Field(self.n // (other % 2)), Z2Field(0)
        else:
            raise Z2DomainError("You are trying to divide Z2Field by " + repr(other))
            
    def __floordiv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[0]
    
    def __truediv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[0]

    def __mod__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(self, other)[1]
    

    def __rfloordiv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[0]
    
    def __rtruediv__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[0]

    def __rmod__(self, other: 'Z2Field|int') -> 'Z2Field':
        return divmod(other, self)[1]
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __eq__(self, other):
        if isinstance(other, Z2Field):
            return self.n == other.n
        elif isinstance(other, int):
            return self.n == (other % 2)
        else:
            raise Z2DomainError("You are trying to compare Z2Field to " + repr(other))
    
    def __neg__(self):
        return Z2Field(self)
    
    def __sub__(self, other):
        return self.__add__(Z2Field(other).__neg__())
    
    def __int__(self):
        return self.n
    
    def __str__(self):
        return 'Z' + str(self.n)
    


class PolynomialDomainError(ValueError):
    pass


class Polynomial:
    """Многочлен над полем"""
    
    def __init__(self, arg: 'int|list[int]|Polynomial|Z2Field'= 0):
        """
        coefficients -- коэффициенты
        """
        if isinstance(arg, int) or isinstance(arg, Z2Field):
            self.coefficients = [arg]
        elif isinstance(arg, list):
            self.coefficients = arg.copy()
        elif isinstance(arg, Polynomial):
            self.coefficients = arg.coefficients.copy()
        else:
            raise PolynomialDomainError("You are trying to create Polynomial from " + repr(arg))
        self.cleanup(self.coefficients)
            
    @staticmethod
    def cleanup(coefficients: 'list[int]') -> 'None':
        while len(coefficients) and coefficients[-1] == 0:
            del coefficients[-1]


    def __str__(self):
        return (" + ".join([
            str(c) + ("*x^" + str(i) if i > 0 else "")
            for c, i in reversed(list(zip(self.coefficients, itertools.count()))) if c != 0
        ])) if len(self.coefficients) else '0'

    
    def __eq__(self, other: 'Polynomial|int') -> 'bool':
        if isinstance(other, int):
            other = Polynomial(other)
        
        if isinstance(other, Polynomial):
            return self.coefficients == other.coefficients
        else:
            raise PolynomialDomainError("Can't say if Polynomial is equal to " + str(type(other)))

    
    def __lshift__(self, deg: 'int') -> 'Polynomial':
        return Polynomial(([0] * deg) + self.coefficients)
    
    def __add__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)
        
        sc = self.coefficients.copy()
        oc = other.coefficients.copy()
        
        # make lengths equal
        sc += [0] * (len(oc)-len(sc))
        oc += [0] * (len(sc)-len(oc))
        
        return Polynomial([
            (sce + oce) for sce, oce in zip(sc, oc)
        ])
        
        
    def __radd__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__add__(other)  # Коммутативность

    
    def __neg__(self) -> 'Polynomial':
        return Polynomial([-c for c in self.coefficients])

    
    def __sub__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)

        return self.__add__(other.__neg__())

    
    def __rsub__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__neg__().__add__(other)

    
    def __mul__(self, other: 'Polynomial|int') -> 'Polynomial':
        if not isinstance(other, Polynomial):
            other = Polynomial(other)
        
        c = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for sc, sci in zip(self.coefficients, itertools.count()):
            for oc, oci in zip(other.coefficients, itertools.count()):
                c[sci + oci] = (c[sci + oci] + (sc * oc))
        
        return Polynomial(c)
        
        
    def __rmul__(self, other: 'Polynomial|int') -> 'Polynomial':
        return self.__mul__(other)  # Коммутативность

    
    def __divmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        if isinstance(other, int):
            other = Polynomial(other)

        sc = Polynomial(self)
        oc = other
        d = Polynomial()
        
        while len(sc.coefficients) >= len(oc.coefficients) > 0:
            c = sc.coefficients[-1] / oc.coefficients[-1]
            t = (len(sc.coefficients) - len(oc.coefficients))
            sc -= (oc << t) * c
            d += Polynomial([0] * t + [c])
            
        return d, sc
    

    def __floordiv__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(self, other)[0]

    def __mod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(self, other)[1]
    

    def __rfloordiv__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(other, self)[0]

    def __rmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return divmod(other, self)[1]
    
    
    def __rdivmod__(self, other: 'Polynomial|int') -> 'Polynomial':
        return Polynomial(other).__divmod__(self)
    

    def __int__(self) -> 'int':
        if not len(self.coefficients):
            return 0
        elif len(self.coefficients) == 1:
            return complex(self.coefficients[0])
        else:
            raise PolynomialDomainError("Can't consider higher degree Polynomial as an integer")
        
        return int(self.coefficients[0])

def stre(n):
    return str(n) + ' ' * (15 - len(str(n)))
Z0 = Z2Field(0)
Z1 = Z2Field(1)

c = Polynomial([Z2Field(1),Z2Field(1),Z2Field(1)])

print(c % Polynomial([Z2Field(0),Z2Field(1),Z2Field(1)]))
print(c % Polynomial([Z2Field(0),Z2Field(0),Z2Field(1)]))
print(c % Polynomial([Z2Field(1),Z2Field(0),Z2Field(1)]))
print(c % Polynomial([Z2Field(0),Z2Field(1),Z2Field(1)]))
print()

print()

l = [Polynomial([Z2Field(0)]), Polynomial([Z2Field(1)]), Polynomial([Z2Field(0), Z2Field(1)]), Polynomial([Z2Field(1), Z2Field(1)])]

for i in range(1, 4):
    for j in range(1, 4):
        print(stre(l[i]), stre(l[j]), stre((l[i] + l[j]) % c))
        
print()

for i in range(1, 4):
    for j in range(1, 4):
        print(stre(l[i]), stre(l[j]), stre((l[i] * l[j]) % c))

                            
                                                



Z1
Z1*x^1 + Z1
Z1*x^1
Z1

Z1
Z1

Z1              Z1              0              
Z1              Z1*x^1          Z1*x^1 + Z1    
Z1              Z1*x^1 + Z1     Z1*x^1         
Z1*x^1          Z1              Z1*x^1 + Z1    
Z1*x^1          Z1*x^1          0              
Z1*x^1          Z1*x^1 + Z1     Z1             
Z1*x^1 + Z1     Z1              Z1*x^1         
Z1*x^1 + Z1     Z1*x^1          Z1             
Z1*x^1 + Z1     Z1*x^1 + Z1     0              

Z1              Z1              Z1             
Z1              Z1*x^1          Z1*x^1         
Z1              Z1*x^1 + Z1     Z1*x^1 + Z1    
Z1*x^1          Z1              Z1*x^1         
Z1*x^1          Z1*x^1          Z1*x^1 + Z1    
Z1*x^1          Z1*x^1 + Z1     Z1             
Z1*x^1 + Z1     Z1              Z1*x^1 + Z1    
Z1*x^1 + Z1     Z1*x^1          Z1             
Z1*x^1 + Z1     Z1*x^1 + Z1     Z1*x^1         
